In [ ]:
#import functions
import sys,os
import glob
try: 
    if(cwd is not None):
        from functions.utils_curation import *
except:
    %cd ..
    cwd = os.getcwd()
    sys.path.insert(0,cwd)
    from functions.utils_curation import *
    

In [ ]:
## Define file name
#path to file IS NOT NEEDED
#fname = 'single protein format - (IC50only) precuration.csv'

#summary
standardiseLoc = 'standardise_summary.csv'
binaryDupRemov = 'binaryDup_summary.csv'
regressDupRemov = 'regressionDup_summary.csv'
multiclasDupRemov = 'multiclass_summary.csv'

## DATA PREPARATION AND STANDARDISATION

In [ ]:
#load and merge datasets
ds1 = check_extention('cell based format - ic50_CHO.csv', 1)
ds2 = check_extention('cell based format - ic50_HEK.csv', 1)
ds3 = check_extention('single protein format - (IC50only) precuration.csv', 1)

In [ ]:
ds1 = ds1[['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Value', 'Standard Units', 'Standard Relation']]
ds2 = ds2[['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Value', 'Standard Units', 'Standard Relation']]
ds3 = ds3[['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Value', 'Standard Units', 'Standard Relation']]

dflist = [ds1, ds2, ds3]
#concat
df0 = pd.concat(dflist)

In [ ]:
#load_data
#df0 = check_extention(fname, 1)
datasuma = [['initial', len(df0)]]
datasuma = pd.DataFrame(datasuma, columns=['process', 'value'])
datasuma.to_csv(f'./data/data_summary/{standardiseLoc}', index=False)
df0

In [ ]:
#drop missing values
df0 = df0.dropna(subset=['Standard Value'])
datasuma = pd.read_csv(f'./data/data_summary/{standardiseLoc}')
new_row = ['missing activity removed', int(datasuma['value'][0])-len(df0)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df0)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{standardiseLoc}', index=False)

df0 = df0.dropna(subset=['Standard Units'])
datasuma = pd.read_csv(f'./data/data_summary/{standardiseLoc}')
new_row = ['missing units removed', int(datasuma['value'][2])-len(df0)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df0)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{standardiseLoc}', index=False)


df0 = df0.reset_index()
df0 = df0.drop(columns = 'index')

In [ ]:
#check if every activity data is the same
df0.groupby('Standard Type').size()

In [ ]:
#check if every activity data is the same
df0.groupby('Standard Units').size()

In [ ]:
#remove unwanted activity data
diffact = []
DropList = []
for index, activity in enumerate(df0['Standard Units']):
    if str(activity).lower() == 'nm':
        pass
    else:
        diffact.append(df0.iloc[[index]])
        DropList.append(index)

if len(diffact) == 0:
    pass
else:
    df0 = df0.drop(DropList, errors="ignore")
    diffact = pd.concat(diffact)
    diffact.to_csv(f'{errorverbose}different_activity.csv')
    df0 = df0.reset_index()
    df0 = df0.drop(columns = 'index')
    datasuma = pd.read_csv(f'./data/data_summary/{standardiseLoc}')
    new_row = ['different activity removed', int(datasuma['value'][4])-len(df0)]
    datasuma.loc[len(datasuma.index)] = new_row
    datasuma.to_csv(f'./data/data_summary/{standardiseLoc}', index=False)
    new_row = ['total', len(df0)]
    datasuma.loc[len(datasuma.index)] = new_row
    datasuma.to_csv(f'./data/data_summary/{standardiseLoc}', index=False)


In [ ]:
#convert IC50 to pIC50
pIC50 = []
for value in df0['Standard Value']:
    value = value / 1000
    pIC50.append(-(math.log10(value*10**-6)))

df0['pIC50 (uM)'] = pIC50

In [ ]:
#remove unwanted columns
dropList = ['Standard Type', 'Standard Units']
df0 = df0.drop(columns = dropList)
df0

In [ ]:
#rename columns if needed
df0.rename(columns = {'Molecule ChEMBL ID':'ID', 'Smiles':'SMILES', 'Standard Relation':'Relation', 'Standard Value': 'IC50 (uM)'}, inplace = True)
df0

In [ ]:
#structural curation
curated_dataset = curate(df0, errorverbose, save_data)

## DUPLICATE REMOVAL BINARY

In [ ]:
#load standardised data
fname = 'standardised_but_no_duplicates_removed.csv'
df1 = check_extention(fname, 2)

datasuma = [['initial', len(df1)]]
datasuma = pd.DataFrame(datasuma, columns=['process', 'value'])
datasuma.to_csv(f'./data/data_summary/{binaryDupRemov}', index=False)
df1

In [ ]:
#check types of relation
df1.groupby('Relation').size()

In [ ]:
df1 = relationTreat(dataset = df1, relationcolumn = 'Relation', activitycolumn = 'IC50 (uM)', threshold = 10, curationtype = 'binary', errorverbose = errorverbose, summary=binaryDupRemov)
df1

In [ ]:
#define threshold
outcome = [1 if activity > 5 else 0 for activity in df1['pIC50 (uM)']]
df1['Outcome'] = outcome
df1

In [ ]:
#group duplicates based on inchikey
df1_agg = group(df1, ['pIC50 (uM)', 'ID', 'SMILES', 'Outcome'])
datasuma = pd.read_csv(f'./data/data_summary/{binaryDupRemov}')
new_row = ['duplicates total', len(df1) - len(df1_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df1_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{binaryDupRemov}', index=False)
df1_agg


In [ ]:
#remove duplicates with stddev > 0
df1_agg = dupRemovalClassification(df1_agg, errorverbose, 'Outcome', 'binary')
datasuma = pd.read_csv(f'./data/data_summary/{binaryDupRemov}')
new_row = ['discordant duplicates', int(datasuma['value'][9]) - len(df1_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df1_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{binaryDupRemov}', index=False)
df1_agg

In [ ]:
df1 = removeListedValues(df1_agg)
df1

In [ ]:
#save curated data
df1.to_csv(f'{save_data}curated_binary.csv', index=False)

## DUPLICATE REMOVAL REGRESSION

In [ ]:
#load standardised data
fname = 'standardised_but_no_duplicates_removed.csv'
df2 = check_extention(fname, 2)
datasuma = [['initial', len(df2)]]
datasuma = pd.DataFrame(datasuma, columns=['process', 'value'])
datasuma.to_csv(f'./data/data_summary/{regressDupRemov}', index=False)
df2

In [ ]:
#remove relations != "="

index_drop = []
for index, relation in enumerate(df2['Relation']):
    if relation != "'='":
        index_drop.append(index)
    else:
        pass

if len(index_drop) == 0:
    pass
else:
    mask = df2.iloc[index_drop]
    df2 = df2.drop(index_drop, errors="ignore")
    mask.to_csv("{}relationsRemoved_regression.csv".format(errorverbose), sep=',', header=True, index=False)

df2 = df2.reset_index()
df2 = df2.drop(columns = 'index')

datasuma = pd.read_csv(f'./data/data_summary/{regressDupRemov}')
new_row = ["removed relations != '='", int(datasuma['value'][0]) - len(df2)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df2)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{regressDupRemov}', index=False)
        

In [ ]:
#check relations
df2.groupby('Relation').size()

In [ ]:
#Drop relations column
df2 = df2.drop(columns = 'Relation')
df2

In [ ]:
#group duplicates based on inchikey
df2_agg = group(df2, ['pIC50 (uM)', 'ID', 'SMILES'])
datasuma = pd.read_csv(f'./data/data_summary/{regressDupRemov}')
new_row = ['duplicates total', len(df2) - len(df2_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df2_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{regressDupRemov}', index=False)
df2_agg

In [ ]:
df2_agg = dupRemovalRegression(df2_agg, errorverbose, 'pIC50 (uM)', 0.2)
datasuma = pd.read_csv(f'./data/data_summary/{regressDupRemov}')
new_row = ['discordant duplicates', int(datasuma['value'][4]) - len(df2_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df2_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{regressDupRemov}', index=False)
df2_agg

In [ ]:
df2 = removeListedValues(df2_agg)
df2

In [ ]:
#save curated data
df2.to_csv(f'{save_data}curated_regression.csv', index=False)

## DUPLICATE REMOVAL MULTICLASS

In [ ]:
#load standardised data
fname = 'standardised_but_no_duplicates_removed.csv'
df3 = check_extention(fname, 2)
datasuma = [['initial', len(df3)]]
datasuma = pd.DataFrame(datasuma, columns=['process', 'value'])
datasuma.to_csv(f'./data/data_summary/{multiclasDupRemov}', index=False)
df3

In [ ]:
#check types of relation
df3.groupby('Relation').size()

In [ ]:
df3 = relationTreat(dataset = df3, relationcolumn = 'Relation', activitycolumn = 'IC50 (uM)', threshold = 10, curationtype = 'multiclass', errorverbose = errorverbose, summary=multiclasDupRemov)
df3

In [ ]:
#define threshold
outcome = []

for activity in df3['pIC50 (uM)']:
    #nonblocker
    if activity < 4.5:
        outcome.append(0)
    elif activity < 5 and activity >= 4.5:
        outcome.append(1)
    elif activity >= 5 and activity < 6:
        outcome.append(2)
    else:
        outcome.append(3)
df3['Outcome'] = outcome
df3

In [ ]:
#group duplicates based on inchikey
df3_agg = group(df3, ['pIC50 (uM)', 'ID', 'SMILES', 'Outcome'])
datasuma = pd.read_csv(f'./data/data_summary/{multiclasDupRemov}')
new_row = ['duplicates total', len(df3) - len(df3_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df3_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{multiclasDupRemov}', index=False)
df3_agg

In [ ]:
#remove duplicates with stddev > 0
df3_agg = dupRemovalClassification(df3_agg, errorverbose, 'Outcome', 'multiclass')
datasuma = pd.read_csv(f'./data/data_summary/{multiclasDupRemov}')
new_row = ['discordant duplicates', int(datasuma['value'][4]) - len(df3_agg)]
datasuma.loc[len(datasuma.index)] = new_row
new_row = ['total', len(df3_agg)]
datasuma.loc[len(datasuma.index)] = new_row
datasuma.to_csv(f'./data/data_summary/{multiclasDupRemov}', index=False)
df3_agg

In [ ]:
df3 = removeListedValues(df3_agg)
df3

In [ ]:
#save curated data
df3.to_csv(f'{save_data}curated_multiclass.csv', index=False)